In [2]:
import numpy as np
import tensorflow as tf


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [40]:

# get data from train_1.csv
tmp = np.genfromtxt("train_1.csv", dtype=float, delimiter=",")

tem = tmp[1:,2:]
ave = np.mean(tem,axis = 0)*np.ones((891,6))
ptp = np.ptp(tem,axis = 0)*np.ones((891,6))
tem = np.subtract(tem,ave)
tem = np.true_divide(tem,ptp)
tmp[1:,2:] = tem

train_data = tmp[1:601,2:]
validation_data0 = tmp[601:,2:]

train_lable = tmp[1:601,1]
validation_lable0 = tmp[601:,1]
train_lable = train_lable.reshape(600,1)
validation_lable0 = validation_lable0.reshape(291,1)

def add_layer(input,in_size,out_size):
    theta = tf.Variable(tf.random_normal([in_size,out_size]))
    if out_size==1:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
    else:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
        a_0 = tf.ones([600,1])
        a = tf.concat([a_0,a],1)
    return theta,a,z


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32,[None,6])
ys = tf.placeholder(tf.float32,[None,1])

# add hidden layer
theta_1,l1,z_1 = add_layer(xs,6,12)
# add output layer
theta_2,output,z_2 = add_layer(l1,13,1)

#regularzation
lamda = 0.046
regularzation = (tf.reduce_sum(tf.square(theta_1))+tf.reduce_sum(tf.square(theta_2)))*lamda/600

# the error between prediction and real data
J_theta = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = ys,logits = z_2)) + regularzation

a = 3 # learning rate
train_step = tf.train.GradientDescentOptimizer(a).minimize(J_theta)

#prediction
validation_data = tf.constant(validation_data0,tf.float32)
validation_lable = tf.constant(validation_lable0,tf.float32)

a_1 = tf.sigmoid(tf.matmul(validation_data,theta_1))
a_0 = tf.ones([291,1])
a_20 = tf.concat([a_0,a_1],1)
a_2 = tf.sigmoid(tf.matmul(a_20,theta_2))
prediction = tf.round(a_2)

validation = tf.subtract(prediction,validation_lable)
validation = tf.abs(validation)
accuracy =tf.reduce_sum(validation) 
accuracy = (291-accuracy)/291

#initialize
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(40000):
    #training
    sess.run(train_step,feed_dict={xs:train_data,ys:train_lable})
    
    if i%1000 == 0:
        loss = sess.run(J_theta,feed_dict={xs:train_data,ys:train_lable})
        print("J_theta:",loss)
        acc = sess.run(accuracy)
        print(acc)
        




J_theta: 1.3881774
0.63573885
J_theta: 0.46720302
0.7938144
J_theta: 0.4655889
0.7938144
J_theta: 0.4647965
0.7938144
J_theta: 0.46417993
0.7938144
J_theta: 0.46274722
0.7938144
J_theta: 0.45999104
0.8006873
J_theta: 0.45857435
0.8041237
J_theta: 0.45775956
0.80756015
J_theta: 0.457312
0.8178694
J_theta: 0.45708436
0.8213059
J_theta: 0.45695385
0.82474226
J_theta: 0.45686704
0.82474226
J_theta: 0.4568075
0.82474226
J_theta: 0.45676786
0.82474226
J_theta: 0.45674196
0.82474226
J_theta: 0.45672455
0.82474226
J_theta: 0.45671222
0.82474226
J_theta: 0.45670298
0.82474226
J_theta: 0.4566958
0.8213059
J_theta: 0.45668995
0.8213059
J_theta: 0.45668504
0.8213059
J_theta: 0.45668098
0.8213059
J_theta: 0.45667753
0.8213059
J_theta: 0.45667464
0.8213059
J_theta: 0.4566722
0.8213059
J_theta: 0.4566702
0.8213059
J_theta: 0.45666853
0.8213059
J_theta: 0.45666713
0.8213059
J_theta: 0.45666602
0.8213059
J_theta: 0.4566651
0.8213059
J_theta: 0.45666438
0.8213059
J_theta: 0.45666376
0.8213059
J_theta: 0

In [42]:
test = np.genfromtxt("test_1.csv",dtype = float,delimiter=",")
test

tem1 = test[1:,1:]
ave1 = np.mean(tem1,axis = 0)*np.ones((418,6))
ptp1 = np.ptp(tem1,axis = 0)*np.ones((418,6))
tem1 = np.subtract(tem1,ave1)
tem1 = np.true_divide(tem1,ptp1)
test[1:,1:] = tem1
test = test[1:,1:]

test_data = tf.constant(test,tf.float32)


t_1 = tf.sigmoid(tf.matmul(test_data,theta_1))
t_0 = tf.ones([418,1])
t_20 = tf.concat([t_0,t_1],1)
t_2 = tf.sigmoid(tf.matmul(t_20,theta_2))
prediction_1 = tf.round(t_2)
result = sess.run(prediction_1)
np.savetxt("result_1.csv", result, delimiter=",")
